# Prepare data

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import re

### Load original datasets

In [2]:

os.chdir("/Users/annaolsen/Desktop/Speciale/DS_thesis/data")
print(os.getcwd())

# Load datasets
df_bio = pd.read_csv("Tara_Biodiversity.csv")
df_meso = pd.read_csv("Tara_Env_Meso_SampleLocation.csv")
df_nutri = pd.read_csv("Tara_Env_Nut.csv")

# New column names for Meso SL
new_column_names = {
    'Sample ID (TARA_barcode#, registered at ...)': 'Sample ID (TARA_barcode#)',
    'Station (TARA_station#, registered at ...)': 'Station'
}

# Rename the columns
df_meso = df_meso.rename(columns=new_column_names)



/Users/annaolsen/Desktop/Speciale/DS_thesis/data


### Sample IDs
Remove unique sample IDs from BIO 

In [3]:
# Find unique sample IDs
ids_df_bio = set(df_bio["Sample ID (TARA_barcode#)"])
ids_df_meso = set(df_meso["Sample ID (TARA_barcode#)"])
ids_df_nutri = set(df_nutri['Sample ID'])

# Find the Sample IDs that are not common between the DataFrames
unique_ids_bio = ids_df_bio - ids_df_meso
unique_ids_meso = ids_df_meso - ids_df_bio
unique_ids_nutri = ids_df_nutri - ids_df_meso

print("Unique IDs bio:", len(unique_ids_bio))
print("Unique IDs meso:", len(unique_ids_meso))
print("Unique IDs nutri:", len(unique_ids_nutri))
print()
print("Original shape of bio:", df_bio.shape)
print("Original shape of meso:", df_meso.shape)
print("Original shape of nutri:", df_nutri.shape)
print()

# Filter unique IDs from df_bio
unique_df_bio = df_bio[df_bio["Sample ID (TARA_barcode#)"].isin(
    unique_ids_bio)]

# Filter unique IDs from df_meso
unique_df_meso = df_meso[df_meso["Sample ID (TARA_barcode#)"].isin(
    unique_ids_meso)]

# Display the lengths of the unique dataframes
print("Shape of unique bio df:", unique_df_bio.shape)
print("Shape of unique meso df:", unique_df_meso.shape)

# Create a boolean mask where IDs are in the list of IDs to remove
mask = df_bio['Sample ID (TARA_barcode#)'].isin(unique_ids_bio)

# Invert the mask to keep the rows that are not in the list of IDs to remove
df_bio = df_bio[~mask].reset_index(drop=True)

print("Shape (bio) after removing unique IDs:", df_bio.shape)


Unique IDs bio: 73
Unique IDs meso: 81
Unique IDs nutri: 0

Original shape of bio: (34768, 19)
Original shape of meso: (34776, 71)
Original shape of nutri: (34776, 38)

Shape of unique bio df: (73, 19)
Shape of unique meso df: (81, 71)
Shape (bio) after removing unique IDs: (34695, 19)


### Merge Bio and Meso

In [4]:
df_bio = df_bio.sort_values(by='Sample ID (TARA_barcode#)')
df_meso = df_meso.sort_values(by='Sample ID (TARA_barcode#)')

columns_to_drop = ['Campaign', 'Station (TARA_station#)', 'Event',
                   'Env feature (abbreviation)',
                   'Env feature (full name (ENVO:ID), terms re...)'
                   ]

df_bio = df_bio.drop(columns=columns_to_drop)

# Columns we don't want to duplicate
duplicate_cols = ['Sample ID (TARA_barcode#)']

# Merge df_bio and df_meso
df_merged = pd.merge(df_meso, df_bio, on=duplicate_cols, how="outer")

print("Shape after merging bio and meso:", df_merged.shape)


Shape after merging bio and meso: (34776, 84)


### Merge Bio-Meso with Nutri

In [5]:
cols_to_drop_nutri = [
    'Sample ID (Bio)', 'Sample ID (ENA)', 'Station', 'Event',
    'Env feature', 'Depth nominal', 'Depth top/min', 'Depth bot/max',
    'Size frac lower', 'Size frac upper', 'Sample material',
    'Sample method', 'Sample code/label', 'File name', 'Distance',
    'Duration(ISO)', 'Number of observations', 'Nitrite min',
    'Nitrite lower', 'Nitrite median', 'Nitrite upper', 'Nitrite max',
    'Nitrate/Nitrite min', 'Nitrate/Nitrite lower', 'Nitrate/Nitrite median',
    'Nitrate/Nitrite upper', 'Nitrate/Nitrite max', 'Silicate min',
    'Silicate lower', 'Silicate median', 'Silicate upper', 'Silicate max'
]

df_nutri = df_nutri.drop(columns=cols_to_drop_nutri)


# Rename the columns
df_merged = df_merged.rename(
    columns={'Sample ID (TARA_barcode#)': 'Sample ID'})


df_merged = pd.merge(df_merged, df_nutri, on='Sample ID', how="inner")

print("Shape after merging bio/meso with nutri:", df_merged.shape)

Shape after merging bio/meso with nutri: (34776, 89)


In [6]:
print(df_merged['Env feature ([abbreviation], full name (EN...)'].value_counts())


Env feature ([abbreviation], full name (EN...)
[SRF] surface water layer (ENVO:00010504)                                               16736
[DCM] deep chlorophyll maximum layer (ENVO:01000326)                                     8476
[ZZZ] marine water layer (ENVO:01000295)                                                 5347
[MES] marine water layer (ENVO:01000295) within the mesopelagic zone (ENVO:00000213)     3271
[MIX] marine epipelagic wind mixed layer (ENVO:01000061)                                  921
unknown                                                                                    13
[FSW] filtered sea water, used to control protocols                                        12
Name: count, dtype: int64


### Data cleaning

In [7]:

os.chdir("/Users/annaolsen/Desktop/Speciale/DS_thesis/data")
print(os.getcwd())

# Load merged dataset
# df = pd.read_csv("Tara_merged_BMN.csv")


# --------- Column names ---------
df = df_merged.copy()

# List of new column names
new_column_names = [
    'Sample ID', 'Sample ID (Bio)', 'Sample ID (ENA)', 'Basis', 'Campaign',
    'Station', 'Method/Device', 'Event', 'Date/Time', 'Latitude', 'Longitude',
    'Depth Layer Zone', 'Depth nominal', 'Depth top', 'Depth bot',
    'Frac lower', 'Frac upper', 'Sample material', 'Sample method',
    'Sample label', 'MP biome', 'OS region', 'BG province',
    'Sea ice conc', 'Sea ice free start', 'Sea ice free days',
    'Sea ice free end', 'Season', 'Season (early)',
    'Moon phase nom', 'Moon phase prop', 'Time of day', 'Sunshine duration',
    'Radiation', 'Radiation 8.1', 'Radiation 8.2', 'Radiation 30',
    'Sea Surface Temp', 'Iron', 'Iron std', 'Ammonium', 'Ammonium std',
    'Nitrite', 'Nitrite std', 'Nitrate', 'Nitrate std',
    'Sea surface fluorescence', 'Sea surface chlorophyll a',
    'Chlorophyll a', 'Sea surface quantum fluorescence',
    'Net PP carbon', 'Net PP carbon 30', 'Total suspended matter',
    'Particulate Organic Carbon', 'Particulate Inorganic Carbon',
    'Depth bathy', 'Latitude.1', 'Longitude.1', 'Distance closest',
    'Sea surface temp grad', 'Strain sub-mesoscale', 'u', 'v',
    'Okubo-Weiss', 'Max Lyapunov Exp', 'Residence time',
    'Latitude.2', 'Longitude.2', 'Age', 'Latitude.3', 'Longitude.3',
    'Shannon_Darwin_mean_all', 'Shannon_Darwin_month_all',
    'Shannon_Darwin_mean_grp', 'Shannon_Darwin_month_grp',
    'Shannon_Physat_month', 'Shannon_Physat_mean', 'SILVA_Chao', 'SILVA_ace',
    'SILVA_Shannon', 'SILVA_species_rich', 'SILVA_func_diversity',
    'Functional richness', 'Functional evenness', 'Phosphate min', 'Phosphate lower',
    'Phosphate median', 'Phosphate upper', 'Phosphate max'
]

# Change column names to list of new column names
df.columns = new_column_names

# --------- Add columns ---------

# Convert the 'Date/Time' column to datetime format
df['Date/Time'] = pd.to_datetime(df['Date/Time'])

# Extract Date and Year into new columns
df['Date'] = df['Date/Time'].dt.date
df['Year'] = df['Date/Time'].dt.year

# Function to extract substrings within brackets [MES], [SRF] etc.
def extract_code(value):
    match = re.search(r'\[([A-Z]+)\]', value)
    if match:
        return match.group(1)
    else:
        return value

# Use function to make new column with Depth Layer abbreviations 
df['Depth Layer'] = df['Depth Layer Zone'].apply(extract_code)


# --------- Data cleaning ---------

# Remove the ID pattern from columns - Example: (MRGID:1905)
ID_pattern = r'\(MRGID:(\d+)\)'

df['OS region'] = df['OS region'].str.replace(ID_pattern, '', regex=True)
df['BG province'] = df['BG province'].str.replace(ID_pattern, '', regex=True)


# Remove the ENVO pattern from column - Example: (ENVO:00010504)
ENVO_pattern = r'\(ENVO:(\d+)\)'

df['Depth Layer Zone'] = df['Depth Layer Zone'].str.replace(
    ENVO_pattern, '', regex=True)

# Remove the text in brackets from columns - Examples: [SRF] and [MS]
bracket_columns = ['Depth Layer Zone', 'BG province', 'OS region']

for col in bracket_columns:
    df[col] = df[col].str.split("] ", expand=True)[1]  # Keep text after "] "
    df[col] = df[col].str.strip()  # Strip for trailing whitespace


# Replace values in Depth Layer Zone ('original': 'replace with')
depth_layer_replace = {
    'surface water layer': 'Surface Water',
    'deep chlorophyll maximum layer': 'Deep Chlorophyll Maximum',
    'marine water layer': 'Marine Water',
    'marine water layer  within the mesopelagic zone': 'Marine Water Mesopelagic Zone',
    'marine epipelagic wind mixed layer': 'Marine Epipelagic Wind Mixed',
    'filtered sea water, used to control protocols': 'Filtered Sea Water',
}

# Apply replacements to Depth Layer Zone
df['Depth Layer Zone'] = df['Depth Layer Zone'].replace(depth_layer_replace)


# --------- Drop columns and sort dataframe ---------

cols_to_drop = [
    'Age',
    'Sample ID (Bio)', 'Sample ID (ENA)', 
    'Season (early)', 'Moon phase nom', 'Moon phase prop',
    'Latitude.1', 'Longitude.1', 'Distance closest',
    'Latitude.2', 'Longitude.2', 'Latitude.3', 'Longitude.3',
    'Phosphate lower', 'Phosphate upper']

df = df.drop(cols_to_drop, axis=1)

# Sort values
df = df.sort_values(by='Date/Time')

# Sort all columns (but Sample ID) alphabetically
columns_to_sort = sorted(
    [col for col in df.columns if col not in ['Sample ID']])

# Reorder the columns
desired_order = ['Sample ID'] + columns_to_sort

# Create a new dataframe with the desired column order, sorted alphabetically
df = df[desired_order]

# Save dataframe to a new CSV file
df.to_csv("Tara_BMN_Cleaned.csv", index=False)

print(df.shape)

/Users/annaolsen/Desktop/Speciale/DS_thesis/data
(34776, 77)


### Remove duplicates

In [8]:

# df = pd.read_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/Tara_BMN_Cleaned.csv")

print("Original DataFrame shape:", df.shape)

columns_remove = [
    'Sample ID', 
    'Frac lower',
    'Frac upper',
    'Sample label', 
    'Sample method',
    'Sample material', 

    # 'Date/Time', 
    # 'Depth Layer Zone',
    # 'Depth bot', 'Depth nominal', 'Depth top',
    # 'Duration (ISO)', 
    # 'Event', 
    # 'Method/Device', 

    ]

# # List of columns to consider when checking for duplicates
columns_to_check = [col for col in df.columns if col not in columns_remove]

# Drop duplicates based on specified columns
df_unique = df.drop_duplicates(subset=columns_to_check)

print(f"Unique DataFrame: {df_unique.shape}")

# Drop rows without values in "Latitude" and "Longitude"
df_unique = df_unique.dropna(subset=['Longitude', 'Latitude'])

# Sort values and reset index
df_unique = df_unique.sort_values(by='Date/Time').reset_index(drop=True)

print(f"Unique DataFrame with Latitude and Longitude: {df_unique.shape}")

df_unique.to_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/TARA_Dash.csv", index=False)

Original DataFrame shape: (34776, 77)
Unique DataFrame: (4486, 77)
Unique DataFrame with Latitude and Longitude: (4474, 77)


#### Unique rows in bio/depth data

In [10]:
df_bio = pd.read_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/Tara_Biodiversity.csv")
df_depth = pd.read_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/Tara_Environmental_Depth.csv")

# print(df_bio.shape)
# print(df_depth.shape)

# Columns we don't want to duplicate
duplicate_cols = ['Sample ID (TARA_barcode#)', 'Campaign',
                  'Station (TARA_station#)', 'Event',
                  'Env feature (abbreviation)',
                  'Env feature (full name (ENVO:ID), terms re...)']

# Merge df_bio and df_depth
df_merged = pd.merge(df_bio, df_depth, on=duplicate_cols, how="inner")

print("Shape after merging bio and depth:", df_merged.shape)


columns_remove = ['Sample ID (TARA_barcode#)']

# List of columns to consider when checking for duplicates
columns_to_check = [col for col in df_merged.columns if col not in columns_remove]

# Drop duplicates based on specified columns
unique_rows = df_merged.drop_duplicates(subset=columns_to_check)

# Count the number of unique rows
num_unique_rows = len(unique_rows)

print("Number of unique rows:", num_unique_rows)

Shape after merging bio and depth: (34768, 76)
Number of unique rows: 2921


### Process data (TARA_mhws_Dash)

In [10]:
df = pd.read_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/TARA_mhws_Dash_2.csv")

print(df.shape)

cols_to_drop = [
    # 'Sample ID', 
    'Frac lower',
    'Frac upper',
    'Sample label', 
    'Sample method',
    'Sample material', 
    'Date',
    'Year',
    'Depth Layer', 
    'MHW_count', 'MHW_category',
    ]

df_dt = df.drop(cols_to_drop, axis=1)

# print(df_dt.columns)

print(df_dt.shape)


new_column_names = [
    'Sample ID', 
    'Ammonium', 'Ammonium std', 'BG province', 'Basis', 'Campaign',
       'Chlorophyll a', 'Date/Time', 'Depth Layer Zone',
       'Depth bathy', 'Depth bot', 'Depth nominal', 'Depth top', 'Event',
       'Functional evenness', 'Functional richness', 'Iron', 'Iron std',
       'Latitude', 'Longitude', 'MP biome', 'Max Lyapunov Exp',
       'Method/Device', 'NPP Carbon', 'NPP Carbon 30', 'Nitrate',
       'Nitrate std', 'Nitrite', 'Nitrite std', 'OS region', 'Okubo-Weiss',
       'Particulate Inorganic Carbon', 'Particulate Organic Carbon',
       'Phosphate max', 'Phosphate median', 'Phosphate min', 'Radiation',
       'Radiation 30', 'Radiation 8.1', 'Radiation 8.2', 'Residence time',
       'SILVA_Chao', 'SILVA_Shannon', 'SILVA_ace', 'SILVA_func_diversity',
       'SILVA_species_rich', 'Sea Surface Temp', 'Sea ice conc',
       'Sea ice free days', 'Sea ice free end', 'Sea ice free start',
       'SS Chlorophyll a', 'SS Fluorescence',
       'SS Quantum Fluorescence', 'SS Temp Grad', 'Season',
       'Shannon_D_mean_all', 'Shannon_D_mean_grp',
       'Shannon_D_month_all', 'Shannon_D_month_grp',
       'Shannon_P_mean', 'Shannon_P_month', 'Station',
       'Strain sub-mesoscale', 'Sunshine duration', 'Time of day',
       'Total suspended matter', 'u', 'v',
       ]



df_dt.columns = new_column_names

print(df_dt.columns)

df_dt.to_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/TARA_BMN_thesis.csv", index=False)


(4474, 79)
(4474, 69)
Index(['Sample ID', 'Ammonium', 'Ammonium std', 'BG province', 'Basis',
       'Campaign', 'Chlorophyll a', 'Date/Time', 'Depth Layer Zone',
       'Depth bathy', 'Depth bot', 'Depth nominal', 'Depth top', 'Event',
       'Functional evenness', 'Functional richness', 'Iron', 'Iron std',
       'Latitude', 'Longitude', 'MP biome', 'Max Lyapunov Exp',
       'Method/Device', 'NPP Carbon', 'NPP Carbon 30', 'Nitrate',
       'Nitrate std', 'Nitrite', 'Nitrite std', 'OS region', 'Okubo-Weiss',
       'Particulate Inorganic Carbon', 'Particulate Organic Carbon',
       'Phosphate max', 'Phosphate median', 'Phosphate min', 'Radiation',
       'Radiation 30', 'Radiation 8.1', 'Radiation 8.2', 'Residence time',
       'SILVA_Chao', 'SILVA_Shannon', 'SILVA_ace', 'SILVA_func_diversity',
       'SILVA_species_rich', 'Sea Surface Temp', 'Sea ice conc',
       'Sea ice free days', 'Sea ice free end', 'Sea ice free start',
       'SS Chlorophyll a', 'SS Fluorescence', 'SS Quantu

In [11]:
df = pd.read_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/TARA_mhws_Dash_2.csv")

new_column_names = [
    'Sample ID', 'Ammonium', 'Ammonium std', 'BG province', 'Basis',
       'Campaign', 'Chl a', 'Date', 'Date/Time', 'Depth Layer',
       'Depth Layer Zone', 'Depth bathy', 'Depth bot', 'Depth nom',
       'Depth top', 'Event', 'Frac lower', 'Frac upper', 'Functional evenness',
       'Functional richness', 'Iron', 'Iron std', 'Latitude', 'Longitude',
       'MP biome', 'Max Lyapunov Exp', 'Method/Device', 'NPP Carbon',
       'NPP Carbon 30', 'Nitrate', 'Nitrate std', 'Nitrite', 'Nitrite std',
       'OS region', 'Okubo-Weiss', 'PIC', 'POC', 'Phosphate max', 'Phosphate med',
       'Phosphate min', 'Radiation', 'Radiation 30', 'Radiation 8.1',
       'Radiation 8.2', 'Residence time', 'SILVA_Chao', 'SILVA_Shannon',
       'SILVA_ace', 'SILVA_func_diversity', 'SILVA_species_rich',
       'Sample label', 'Sample material', 'Sample method', 'SST',
       'Sea ice conc', 'Sea ice free days', 'Sea ice free end',
       'Sea ice free start', 'SS Chl a', 
       'SS Fluorescence', 'SS Quantum Fluorescence',
       'SST Grad', 'Season', 'Shannon_Darwin_mean_all',
       'Shannon_Darwin_mean_grp', 'Shannon_Darwin_month_all',
       'Shannon_Darwin_month_grp', 'Shannon_Physat_mean',
       'Shannon_Physat_month', 'Station', 'Strain sub-mesoscale',
       'Sunshine duration', 'Time of day', 'TSM', 'Year',
       'u', 'v', 'MHW_count', 'MHW_category']



new_column_names2 = [
    'Sample ID', 'Ammonium', 'Ammonium std', 'BG province', 'Basis',
       'Campaign', 'Chlorophyll a', 'Date', 'Date/Time', 'Depth Layer',
       'Depth Layer Zone', 'Depth bathy', 'Depth bot', 'Depth nominal',
       'Depth top', 'Event', 'Frac lower', 'Frac upper', 'Functional evenness',
       'Functional richness', 'Iron', 'Iron std', 'Latitude', 'Longitude',
       'MP biome', 'Max Lyapunov Exp', 'Method/Device', 'Net PP carbon',
       'Net PP carbon 30', 'Nitrate', 'Nitrate std', 'Nitrite', 'Nitrite std',
       'OS region', 'Okubo-Weiss', 'Particulate Inorganic Carbon',
       'Particulate Organic Carbon', 'Phosphate max', 'Phosphate median',
       'Phosphate min', 'Radiation', 'Radiation 30', 'Radiation 8.1',
       'Radiation 8.2', 'Residence time', 'SILVA_Chao', 'SILVA_Shannon',
       'SILVA_ace', 'SILVA_func_diversity', 'SILVA_species_rich',
       'Sample label', 'Sample material', 'Sample method', 'Sea Surface Temp',
       'Sea ice conc', 'Sea ice free days', 'Sea ice free end',
       'Sea ice free start', 'Sea surface chlorophyll a',
       'Sea surface fluorescence', 'Sea surface quantum fluorescence',
       'Sea surface temp grad', 'Season', 'Shannon_Darwin_mean_all',
       'Shannon_Darwin_mean_grp', 'Shannon_Darwin_month_all',
       'Shannon_Darwin_month_grp', 'Shannon_Physat_mean',
       'Shannon_Physat_month', 'Station', 'Strain sub-mesoscale',
       'Sunshine duration', 'Time of day', 'Total suspended matter', 'Year',
       'u', 'v', 'MHW_count', 'MHW_category']

print(len(new_column_names))
print(len(new_column_names2))


df.columns = new_column_names

print(df.columns)

df.to_csv("/Users/annaolsen/Desktop/Speciale/DS_thesis/data/TARA_mhws_Dash_3.csv", index=False)

79
79
Index(['Sample ID', 'Ammonium', 'Ammonium std', 'BG province', 'Basis',
       'Campaign', 'Chl a', 'Date', 'Date/Time', 'Depth Layer',
       'Depth Layer Zone', 'Depth bathy', 'Depth bot', 'Depth nom',
       'Depth top', 'Event', 'Frac lower', 'Frac upper', 'Functional evenness',
       'Functional richness', 'Iron', 'Iron std', 'Latitude', 'Longitude',
       'MP biome', 'Max Lyapunov Exp', 'Method/Device', 'NPP Carbon',
       'NPP Carbon 30', 'Nitrate', 'Nitrate std', 'Nitrite', 'Nitrite std',
       'OS region', 'Okubo-Weiss', 'PIC', 'POC', 'Phosphate max',
       'Phosphate med', 'Phosphate min', 'Radiation', 'Radiation 30',
       'Radiation 8.1', 'Radiation 8.2', 'Residence time', 'SILVA_Chao',
       'SILVA_Shannon', 'SILVA_ace', 'SILVA_func_diversity',
       'SILVA_species_rich', 'Sample label', 'Sample material',
       'Sample method', 'SST', 'Sea ice conc', 'Sea ice free days',
       'Sea ice free end', 'Sea ice free start', 'SS Chl a', 'SS Fluorescence',
       